In [1]:
import pandas as pd
name = 'SBIN'


In [2]:
trade = pd.read_parquet(f'{name}_trade.parquet')
order = pd.read_parquet(f'{ name}_order.parquet')

In [3]:
order.head()

,Timestamp,Side,Price,Volume_Disclosed,Price_Type,Symbol,Original_Qty,Order Number,IOC Flag,Activity Type
0,79631194614632,B,258.85,0.0,Limit,SBIN,1.0,1300000000002815,N,1
1,79631194618744,S,256.00,0.0,Limit,SBIN,1.0,1300000000002919,N,1
2,79631194632638,S,260.00,0.0,Limit,SBIN,200.0,1300000000003115,N,1
3,79631194641354,B,252.00,0.0,Limit,SBIN,22.0,1300000000003281,N,1
4,79631194673336,S,265.00,0.0,Limit,SBIN,300.0,1300000000003440,N,1


In [4]:
trade.head()

,trade_time,price,quantity,buy_order_number,sell_order_number
7398209,79631223195044,259.6,25,1300000000053655,1300000000057907
7398210,79631223195045,259.6,5,1300000000050079,1300000000057907
7398211,79631223195046,259.6,4,1300000000050265,1300000000057907
7398212,79631223195047,259.6,3,1300000000050535,1300000000057907
7398213,79631223195048,259.6,2,1300000000051136,1300000000057907


In [5]:
avg = trade['price'].mean()


In [6]:
print(avg)

279.14944939175115


In [7]:
import pandas as pd
import csv

def create_quote_stream_corrected(
    order_log_path=f'{name}_order.parquet',
    trade_log_path=f'{name}_trade.parquet',
    quote_output_path=f'{name}_taq.csv'
):
    """
    Corrected version to process logs and create a quote stream.
    Implements robust "delete-then-add" logic for modified orders to prevent
    a corrupted live book and ensure all events are processed correctly.
    """
    # --- Step 1: Loading and Preparing Data ---
    print("--- Step 1: Loading and Preparing Data ---")
    try:
        order_df = pd.read_parquet(order_log_path)
        trade_df = pd.read_parquet(trade_log_path)
        
        for col in ['Order Number', 'buy_order_number', 'sell_order_number']:
            if col in order_df.columns: order_df[col] = order_df[col].astype(str)
            if col in trade_df.columns: trade_df[col] = trade_df[col].astype(str)
    except FileNotFoundError as e:
        print(f"\nError: Data file not found. Make sure '{e.filename}' is in the correct directory.")
        return

    order_df_copy = order_df[order_df['Price_Type'] != 'Trigger'].copy()
    order_df_copy.rename(columns={'Timestamp': 'timestamp', 'Order Number': 'order_number'}, inplace=True)
    def activity_to_event(act_type):
        if act_type == 3: return 'CANCEL'
        if act_type == 4: return 'MODIFY'
        return 'NEW'
    order_df_copy['event_type'] = order_df_copy['Activity Type'].apply(activity_to_event)
    order_events = order_df_copy[['timestamp', 'event_type', 'order_number', 'Side', 'Price', 'Volume_Disclosed', 'Original_Qty']]

    trade_df_copy = trade_df.copy()
    trade_df_copy.rename(columns={'trade_time': 'timestamp'}, inplace=True)
    trade_df_copy['event_type'] = 'TRADE'
    trade_events = trade_df_copy[['timestamp', 'event_type', 'price', 'quantity', 'buy_order_number', 'sell_order_number']]

    # --- Step 2: Combining and Sorting All Market Events ---
    print("\n--- Step 2: Combining and Sorting All Market Events ---")
    all_events = pd.concat([order_events, trade_events], ignore_index=True, sort=False)
    all_events.sort_values(by='timestamp', inplace=True)
    all_events.reset_index(drop=True, inplace=True)
    total_events = len(all_events)
    print(f"Total chronological events to process: {total_events}")

    # --- Step 3: Reconstructing Order Book with Corrected Logic ---
    live_book = {}
    last_traded_price = 0.0
    flag = 0
    
    with open(quote_output_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['timestamp', 'best_bid', 'depth_bid', 'best_bid_ud', 'depth_bid_ud',  'best_ask', 'depth_ask', 'best_ask_ud', 'depth_ask_ud', 'ltp'])

        print("\n--- Step 3: Reconstructing Order Book and Generating Quote Stream ---")
        for index, event in all_events.iterrows():
            timestamp = event['timestamp']

            
            # Part A: Update Live Book
            if event['event_type'] != 'TRADE':
                diff = abs(event['Price'] - avg)
                diff = 100 * diff / avg
                if diff > 10:
                    continue

            if event['event_type'] == 'NEW' and event['Original_Qty'] >= 1e-6:

                live_book[event['order_number']] = {
                    'side': event['Side'], 'price': event['Price'], 'quantity': event['Original_Qty'], 'disclosed' : event['Volume_Disclosed'], 'initial_d' : event['Volume_Disclosed']
                }
            
            # *** CORRECTED MODIFICATION LOGIC ***
            elif event['event_type'] == 'MODIFY':
                order_num = event['order_number']
                volume = event['Original_Qty']
                if order_num in live_book:
                    del live_book[order_num]

                if volume >= 1e-6:
                    live_book[order_num] = {
                      'side': event['Side'], 'price': event['Price'], 'quantity': event['Original_Qty'], 'disclosed' : event['Volume_Disclosed'], 'initial_d' : event['Volume_Disclosed']
                }

            elif event['event_type'] == 'CANCEL':
                if event['order_number'] in live_book:
                    del live_book[event['order_number']]

            elif event['event_type'] == 'TRADE':
                if not live_book:
                    best_bid, best_ask, best_bid_ud, best_ask_ud = 0.0, 0.0, 0.0, 0.0
                else:
                    bids = [order['price'] for order in live_book.values() if order['side'] == 'B' and order['disclosed'] >= 1e-6]
                    asks = [order['price'] for order in live_book.values() if order['side'] == 'S' and order['disclosed'] >= 1e-6]
                    best_bid = max(bids) if bids else 0.0
                    best_ask = min(asks) if asks else 0.0
                    bids = [order['price'] for order in live_book.values() if order['side'] == 'B']
                    asks = [order['price'] for order in live_book.values() if order['side'] == 'S']
                    best_bid_ud = max(bids) if bids else 0.0
                    best_ask_ud = min(asks) if asks else 0.0

               
                bid_depth = sum(o['disclosed'] for o in live_book.values() if o['side'] == 'B' and o['price'] == best_bid)
                ask_depth = sum(o['disclosed'] for o in live_book.values() if o['side'] == 'S' and o['price'] == best_ask)
                depth_bid_ud = sum(o['quantity'] for o in live_book.values() if o['side'] == 'B' and o['price'] == best_bid_ud)
                depth_ask_ud = sum(o['quantity'] for o in live_book.values() if o['side'] == 'S' and o['price'] == best_ask_ud)

                    
                last_traded_price = event['price']
                trade_qty = event['quantity']
                for order_num_hit in [event['buy_order_number'], event['sell_order_number']]:
                    if order_num_hit in live_book:
                        initial_disclosed_size = live_book[order_num_hit]['initial_d']
                        live_book[order_num_hit]['quantity'] -= trade_qty
                        live_book[order_num_hit]['disclosed'] -= trade_qty
                        if live_book[order_num_hit]['quantity'] <= 1e-6:
                            del live_book[order_num_hit]
                        elif live_book[order_num_hit]['disclosed'] <= 1e-6:
                            remaining_qty = live_book[order_num_hit]['quantity']
                            live_book[order_num_hit]['disclosed'] = min(initial_disclosed_size, remaining_qty)

                writer.writerow([timestamp, best_bid, bid_depth, best_bid_ud, depth_bid_ud,  best_ask, ask_depth, best_ask_ud, depth_ask_ud, last_traded_price])

        
            if (index + 1) % 100000 == 0 or (index + 1) == total_events:
                progress_pct = ((index + 1) / total_events) * 100
                print(f"  Processed {index + 1} / {total_events} events ({progress_pct:.1f}%)", end='\r')

    print(f"\n\nProcessing complete! Quote stream saved to '{quote_output_path}'. 🚀")

if __name__ == '__main__':
    create_quote_stream_corrected()

--- Step 1: Loading and Preparing Data ---

--- Step 2: Combining and Sorting All Market Events ---
Total chronological events to process: 1635525

--- Step 3: Reconstructing Order Book and Generating Quote Stream ---
  Processed 1635525 / 1635525 events (100.0%)

Processing complete! Quote stream saved to 'SBIN_taq.csv'. 🚀


In [8]:
trade_q = trade['quantity'].reset_index()
trade_q = trade_q.drop('index', axis = 1)

trade_q.head()

,quantity
0,25
1,5
2,4
3,3
4,2


In [9]:
len(trade_q)

92643

In [10]:
dff = pd.read_csv(f'{name}_taq.csv')

In [11]:
count = ((dff['ltp'] != dff['best_bid']) & (dff['ltp'] != dff['best_ask']) & (dff['ltp'] != dff['best_bid_ud']) & (dff['ltp'] != dff['best_ask_ud'])).sum()
print(count, len(dff), "mismatched = ", 100 * count / len(dff), '%')

9163 92643 mismatched =  9.89065552713103 %


In [12]:
print(len(dff) == len(trade_q))

True


In [13]:
dff = dff.merge(trade_q, left_index=True, right_index=True, how='inner')
len(dff)


92643

In [14]:
dff.head(60006)

,timestamp,best_bid,depth_bid,best_bid_ud,depth_bid_ud,best_ask,depth_ask,best_ask_ud,depth_ask_ud,ltp,quantity
0,79631223195044,0.00,0.0,282.0,25.0,0.00,0.0,253.60,400.0,259.60,25
1,79631223195045,0.00,0.0,272.5,5.0,0.00,0.0,253.60,400.0,259.60,5
2,79631223195046,0.00,0.0,272.4,4.0,0.00,0.0,253.60,400.0,259.60,4
3,79631223195047,0.00,0.0,272.3,3.0,0.00,0.0,253.60,400.0,259.60,3
4,79631223195048,0.00,0.0,272.2,2.0,0.00,0.0,253.60,400.0,259.60,2
...,...,...,...,...,...,...,...,...,...,...,...
60001,79632318874429,257.25,330.0,260.1,200.0,257.25,4.0,257.25,35.0,257.25,35
60002,79632318874438,257.25,330.0,260.1,200.0,257.30,13.0,257.30,13.0,257.30,13
60003,79632318957182,257.25,330.0,260.1,200.0,257.25,2.0,257.25,12.0,257.25,12
60004,79632319030985,257.25,330.0,260.1,200.0,257.40,1120.0,257.35,976.0,257.35,163


In [15]:
dff.to_parquet(f'{name}_taq.parquet')

117456

FOR TESTING

In [6]:
try:
    order_df = pd.read_parquet(f'{name}_order.parquet')
    trade_df = pd.read_parquet(f'{name}_trade.parquet')
    for col in ['Order Number', 'buy_order_number', 'sell_order_number']:
        if col in order_df.columns: order_df[col] = order_df[col].astype(str)
        if col in trade_df.columns: trade_df[col] = trade_df[col].astype(str)
except FileNotFoundError as e:
    print(f"\nError: Data file not found. Make sure '{e.filename}' is in the correct directory.")
order_df_copy = order_df[order_df['Price_Type'] != 'Trigger'].copy()
order_df_copy.rename(columns={'Timestamp': 'timestamp', 'Order Number': 'order_number'}, inplace=True)
def activity_to_event(act_type):
    if act_type == 3: return 'CANCEL'
    if act_type == 4: return 'MODIFY'
    return 'NEW'
order_df_copy = order_df[order_df['Price_Type'] != 'Trigger'].copy()
order_df_copy.rename(columns={'Timestamp': 'timestamp', 'Order Number': 'order_number'}, inplace=True)
def activity_to_event(act_type):
    if act_type == 3: return 'CANCEL'
    if act_type == 4: return 'MODIFY'
    return 'NEW'
order_df_copy['event_type'] = order_df_copy['Activity Type'].apply(activity_to_event)
order_events = order_df_copy[['timestamp', 'event_type', 'order_number', 'Side', 'Price', 'Volume_Disclosed', 'Original_Qty']]

trade_df_copy = trade_df.copy()
trade_df_copy.rename(columns={'trade_time': 'timestamp'}, inplace=True)
trade_df_copy['event_type'] = 'TRADE'
trade_events = trade_df_copy[['timestamp', 'event_type', 'price', 'quantity', 'buy_order_number', 'sell_order_number']]

# --- Step 2: Combining and Sorting All Market Events ---
print("\n--- Step 2: Combining and Sorting All Market Events ---")
all_events = pd.concat([order_events, trade_events], ignore_index=True, sort=False)
all_events.sort_values(by='timestamp', inplace=True)
all_events.reset_index(drop=True, inplace=True)
total_events = len(all_events)
print(f"Total chronological events to process: {total_events}")



--- Step 2: Combining and Sorting All Market Events ---
Total chronological events to process: 1635525


In [ ]:
print((int(all_events.iloc[40000]['timestamp']) - int(all_events.iloc[0]['timestamp'])))

396727692


In [46]:
fil = all_events[(all_events['order_number'] == '1300000000808886') | (all_events['sell_order_number'] == '1300000000808886')]
fil.head(1000000)

,timestamp,event_type,order_number,Side,Price,Volume_Disclosed,Original_Qty,price,quantity,buy_order_number,sell_order_number
214020,79631377556911,NEW,1300000000808886,S,10835.0,10.0,100.0,NaN,NaN,NaN,NaN
214021,79631377556912,TRADE,NaN,NaN,NaN,NaN,NaN,10837.0,3.0,1300000000288757,1300000000808886
214022,79631377556913,TRADE,NaN,NaN,NaN,NaN,NaN,10836.0,50.0,1300000000337309,1300000000808886
214519,79631378449388,TRADE,NaN,NaN,NaN,NaN,NaN,10835.0,25.0,1300000000812492,1300000000808886
215575,79631379133006,TRADE,NaN,NaN,NaN,NaN,NaN,10835.0,22.0,1300000000815065,1300000000808886


In [16]:
fil = all_events[(all_events['timestamp'] == '79631892960278')]
fil.head(100000000000)

,timestamp,event_type,order_number,Side,Price,Volume_Disclosed,Original_Qty,price,quantity,buy_order_number,sell_order_number


In [ ]:
fil = all_events[(all_events['Price'] == 10750)]
fil.head()

,timestamp,event_type,order_number,Side,Price,Volume_Disclosed,Original_Qty,price,quantity,buy_order_number,sell_order_number
2463,79631253629589,NEW,1300000000068513,B,10750.0,6.0,60.0,NaN,NaN,NaN,NaN


In [44]:
print((count)) 

6547


In [38]:
filtered_order = trade[trade['price'] == 10835]

In [39]:
filtered_order.head()

,trade_time,price,quantity,buy_order_number,sell_order_number
7679055,79631378449388,10835.0,25,1300000000812492,1300000000808886
7680107,79631379133006,10835.0,22,1300000000815065,1300000000808886
7802674,79631465841508,10835.0,3,1300000000815065,1300000000975781
7810729,79631469886446,10835.0,1,1300000001154854,1300000000975781
7846790,79631490512770,10835.0,10,1300000001240288,1300000000975781
